<a href="https://colab.research.google.com/github/en-ashay/ML-codes/blob/master/A3_Problem_Statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the dataset from the following link:

https://drive.google.com/open?id=0B4c_CWtMboiGRHJhWWZMbGpQazFxcHNfUkhnQ1pQU09IeXFR

If you read the dataset by importing pandas library and using pd.read_csv():

In [21]:
import pandas as pd
import numpy as np
import scipy.stats as st

# Checking for possible functions in scipy

In [27]:

dir(st.norm)

# dir(st.norm.pdf) to check details of norm.pdf
#dir(std.norm.logpdf)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_argcheck',
 '_argcheck_rvs',
 '_cdf',
 '_cdf_single',
 '_cdfvec',
 '_construct_argparser',
 '_construct_default_doc',
 '_construct_doc',
 '_ctor_param',
 '_entropy',
 '_fit_loc_scale_support',
 '_fitstart',
 '_get_support',
 '_isf',
 '_logcdf',
 '_logpdf',
 '_logsf',
 '_mom0_sc',
 '_mom1_sc',
 '_mom_integ0',
 '_mom_integ1',
 '_munp',
 '_nnlf',
 '_nnlf_and_penalty',
 '_open_support_mask',
 '_parse_args',
 '_parse_args_rvs',
 '_parse_args_stats',
 '_pdf',
 '_penalized_nnlf',
 '_ppf',
 '_ppf_single',
 '_ppf_to_solve',
 '_ppfvec',
 '_random_state',
 '_reduce_func',
 '_rvs',
 '_

In [3]:
data = pd.read_csv("data.csv")

In [28]:
data.head(3)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


Remove the first column 'id' and the last column 'Unnamed:32' from this dataset:

In [6]:
# Listing all the columns
print(data.columns.tolist())

['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32']


In [7]:
data.drop(labels=[data.columns[0],data.columns[32]],axis=1,inplace=True)

In [8]:
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


The task is to determine the best fitting probability distribution on each column data (Sample Data of the Random Variable) of the dataset, out of the possible choices of probability distributions: Normal(Symmetric), Rayleigh(Skewed) or Binomial. 

So, let's see that how are we going to do that for a single column data and then we have to repeat this process for rest of the columns: 

Step 1: First, we will check whether the Random Variable is Discrete or Continous by checking that how many unique values are there in the single column of the Random Variable and does the Random Variable has string or floating point or integer values? Usually, Discrete Random Variables have either Integer or String values whereas continous random variables have floating point values and also because the values are not sparse (0s or 1s) in continous random variables and they are dense (floating point values) so values don't repeat much and hence the number of unique values are almost equal to the total number of values in the column. As an example, lets take one by one two columns: 'diagnosis' and 'radius_mean': 

In [9]:
diagnosis_unique_values = data['diagnosis'].unique()

In [13]:
data['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [10]:
radius_mean_unique_values = data['radius_mean'].unique()

In [14]:
data['radius_mean'].value_counts()

12.34    4
12.77    3
15.46    3
12.89    3
13.05    3
        ..
12.31    1
18.81    1
13.30    1
23.09    1
18.25    1
Name: radius_mean, Length: 456, dtype: int64

In [15]:
diagnosis_unique_values

array(['M', 'B'], dtype=object)

In [16]:
radius_mean_unique_values

array([17.99 , 20.57 , 19.69 , 11.42 , 20.29 , 12.45 , 18.25 , 13.71 ,
       13.   , 12.46 , 16.02 , 15.78 , 19.17 , 15.85 , 13.73 , 14.54 ,
       14.68 , 16.13 , 19.81 , 13.54 , 13.08 ,  9.504, 15.34 , 21.16 ,
       16.65 , 17.14 , 14.58 , 18.61 , 15.3  , 17.57 , 18.63 , 11.84 ,
       17.02 , 19.27 , 16.74 , 14.25 , 13.03 , 14.99 , 13.48 , 13.44 ,
       10.95 , 19.07 , 13.28 , 13.17 , 18.65 ,  8.196, 12.05 , 13.49 ,
       11.76 , 13.64 , 11.94 , 18.22 , 15.1  , 11.52 , 19.21 , 14.71 ,
       13.05 ,  8.618, 10.17 ,  8.598,  9.173, 12.68 , 14.78 ,  9.465,
       11.31 ,  9.029, 12.78 , 18.94 ,  8.888, 17.2  , 13.8  , 12.31 ,
       16.07 , 13.53 , 18.05 , 20.18 , 12.86 , 11.45 , 13.34 , 25.22 ,
       19.1  , 12.   , 18.46 , 14.48 , 19.02 , 12.36 , 14.64 , 14.62 ,
       15.37 , 13.27 , 13.45 , 15.06 , 20.26 , 12.18 ,  9.787, 11.6  ,
       14.42 , 13.61 ,  6.981,  9.876, 10.49 , 13.11 , 11.64 , 22.27 ,
       11.34 ,  9.777, 12.63 , 14.26 , 10.51 ,  8.726, 11.93 ,  8.95 ,
      

Let's see that in each of the cases, how many unique values are there:

In [17]:
len(diagnosis_unique_values)

2

In [18]:
len(radius_mean_unique_values)

456

As you can see that in our discrete random variable of 'diagnosis', we have very limited values, only two in number and they are string in nature : ['B, 'M']

Whereas, in our continous random variable of 'radius_mean', we can see the numbe rof unique values is huge and it's 456 which is not much smaller than 569, the number of rows in a column. Moreover, the nature or datatype of values is floating point. 

So, lets write this concept in the form of the code: 

In [58]:
data.dtypes

diagnosis                   object
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst            float64
concave points_worst

In [64]:
print(type(data['diagnosis'][0]))
print(type(data['diagnosis']))

<class 'str'>
<class 'pandas.core.series.Series'>


In [76]:
def determine_random_variable_type(column_name):
    
    if (type(data[column_name][0]) == str or type(data[column_name][0]) == int) and ((data[column_name].nunique()) < len(data[column_name])):
        
        return 'discrete'
    
    else:
        
        return 'continous'

The above function will take the name of the single column of the dataset and access the data of that column from the dataset and check the datatype or nature of the values inside that column and also the total number of unique values in that column and check the conditions for the random variable or that specific column being discrete or continous and returns the the type of random variable as strings 'discrete' or 'continous'. 

Now, the next task is to calculate the value of the best estimator of the population parameter of the probability distribution from which a specific column data is being sampled. In case of discrete random variables, it's pretty easy to determine that: If there are only two unique values in the column data, then it's binomial distribution and multinomial if there are more than two unique values. 

But, the real challenge lies in case of continous random variables, where it is difficult to determine that from which continous probability distibution the column data is being sampled. We have two options here: Either Normal (Symmetric) or Rayleigh (Skewed). So, in this case we have to find the best fit of our data on both the distributions and then select the distribution with the BESTEST FIT. 

Well, now the question arrises that how we are going to do that. We know that the Log Likelihood Function Value calculated on the data determines the quality of fit of data on the presumed distribution and if we maximize that value then we get the best fit of a specific probability distribution on the data. so, we first find the best fit of normal distibution on our continous data and then we find the best fit of Rayleigh Distribution on our continous data by maximizing the Log Likelihood Function Value for both the cases and because we have to select that distribution which gives us the bestest fit so we will end up selecting the distribution whose Log Likelihood Function value is maximum among both maxmized Log Likelihood Function Values. 

The problem is that in order to find the Log Likelihood Function value evaluated in case of fit of specific distribution on the data, we need the PDF of that distribution, which we know but we dont know the values of best estimators of population parameters:
\begin{equation}
\mu, \sigma
\end{equation}
of Normal Distribution and best estimator of population parameter, 
\begin{equation}
\sigma (mode)
\end{equation}

of Rayleigh Distribution. 

Now, how do we get the values of the best estimators of population parameters of respective distributions. Well, we know the answer to this question and that is by Maximum Log Likelihood Estimation. We know that the data in each column of this dataset is IID (Independent and Identically Distributed) therefore, for any random column of this dataset, X where X can be 'radius_mean' or 'testure_mean' or anything else, the Log Likelihood function is evaluated as: 

\begin{equation}
\log_e L(\mu,\sigma) = \log_e P(X=x_1\cap X=x_2\cap .........X=x_N)
\end{equation}
For Normal Distribution

\begin{equation}
\log_e L(\mu, \sigma) = \log_e\prod\limits_{i=0}^{N}\left(\frac{1}{\sqrt{2\pi}\sigma}e^\frac{(x_i-\mu)^2}{2\sigma^2}\right)
\end{equation}

\begin{equation}
\log_e L(\mu, \sigma) = \sum\limits_{i=0}^{N}\log_e\left(\frac{1}{\sqrt{2\pi}\sigma}e^\frac{(x_i-\mu)^2}{2\sigma^2}\right)
\end{equation}
Here, the population mean and population standard deviation estimator best estimate for normal distribution can be evaluated by taking the derivative of the above function with respect to $\mu$ and $\sigma$ and when we do that, that is:
\begin{equation}
\frac{\partial\log_e L(\mu, \sigma)}{\partial\mu} = 0
\end{equation}

\begin{equation}
\frac{\partial log_e L(\mu, \sigma)}{\partial\sigma} = 0
\end{equation}

We get,
\begin{equation}
\mu_\text{best,normal} = \frac{\sum\limits_{i=0}^{N}{X_i}}{N}
\end{equation}

And, 

\begin{equation}
\sigma_\text{best,normal} = \sqrt{\frac{\sum\limits_{i=0}^{N}{(x_i-\mu_\text{best,normal})^2}}{N}}
\end{equation}

Which is sample mean and sample standard deviation, therefore these are the best estimates of the population parameters of Normal Distribution for the best fit on our column data. 

Similarly, we can find the values of the best estimators of the population parameter of Rayliegh Distribution by following the above mentioned steps, and we get the following value for the best estimate in case of Rayleigh Distribution:

\begin{equation}
\sigma_\text{best,rayleigh} = \sqrt{\frac{\sum\limits_{i=0}^{N}{x_i^2}}{2N}}
\end{equation}

So, these best estimates of the MVU Estimators of respective population parameters of respective Probability Distributions will maximize the Log Likelihood Function values for respective probability distributions and hence will result in best fit of respective distributions. Now, to determine the bestest fit among both of the distributions, we have to select that distribution for which the Log Likelihood Function is the highest among both already maximized values. 

Now, in our case N = 569, so the best estimates of the MVU estimators of the population parameters of respective probability distributions are given by:

\begin{equation}
\mu_\text{best,normal} = \frac{\sum\limits_{i=0}^{569}{x_i}}{569}
\end{equation}

And, 

\begin{equation}
\sigma_\text{best,normal} = \sqrt{\frac{\sum\limits_{i=0}^{569}{(x_i-\mu_\text{best,normal})^2}}{569}}
\end{equation}

For Normal Distribution

And, 

\begin{equation}
\sigma_\text{best,rayleigh} = \sqrt{\frac{\sum\limits_{i=0}^{569}{x_i^2}}{2*569}}
\end{equation}


Therefore, the maximized values of Log Likelihood Function for the case of both the probability distributions is given by:

\begin{equation}
L_\text{max,normal} = \log_e L(\mu_\text{best,normal},\sigma_\text{best,normal})
\end{equation}

\begin{equation}
L_\text{max,normal} = \sum\limits_{i=0}^{569}\log_e\left(\frac{1}{\sqrt{2\pi}\sigma_\text{best,normal}}e^\frac{(x_i-\mu_\text{best,normal})^2}{2\sigma_\text{best,normal}^2}\right)
\end{equation}

For Normal Distribution

And,

Similarly, maximized value of Log Likelihood Function for Rayleigh Distribution can be calculated as we have just calculated for Normal Distribution. 
\begin{equation}
L_\text{max,rayleigh} = \log_e L(\sigma_\text{best,rayleigh})
\end{equation}

Now, the column data will be considered to be sampled from Normal Distribution if:

\begin{equation}
L_\text{max,normal} > L_\text{max,rayleigh}
\end{equation}

else:

column data will be considered to be sampled from Rayleigh Distribution.

Therefore, in order to do all that, we are going to create following set of functions:

In [48]:
def calculate_L_max_normal(column_name):
    
    mu_best_normal = data[column_name].mean()
    
    sigma_best_normal = data[column_name].std()
    
    L_max_normal = sum(st.norm.logpdf(data[column_name],mu_best_normal,sigma_best_normal))
    
    return L_max_normal

In [50]:
def calculate_L_max_rayleigh(column_name):
    
    sigma_best_rayleigh = np.sqrt(data[column_name]**2/2*(len(data[column_name])))
    
    L_max_rayleigh = sum(st.rayleigh.logpdf(data[column_name],sigma_best_rayleigh))

    return L_max_rayleigh

In [78]:
def determine_distribution_type(column_name):
    
    column_type=determine_random_variable_type(column_name)
    
    if column_type == 'discrete':
        
       distrinution = "bionomial" if (data[column_name].nunique())==2 else "Multinomial" 
       return distrinution 
    else:
        # negative log likelihood
        L_max_normal  = -(calculate_L_max_normal(column_name))
        
        L_max_rayleigh  = -(calculate_L_max_rayleigh(column_name))
        
        if L_max_normal > L_max_rayleigh:
            
            return 'normal'
        
        else:
            
            return 'rayleigh'

In [79]:
determine_distribution_type('diagnosis')

'bionomial'

# To print details of all columns 

In [98]:
for columns in data.columns:
    print(columns,'\t','=',[determine_random_variable_type(columns)],[determine_distribution_type(columns)],'\n')

diagnosis 	 = ['discrete'] ['bionomial'] 

radius_mean 	 = ['continous'] ['rayleigh'] 

texture_mean 	 = ['continous'] ['rayleigh'] 

perimeter_mean 	 = ['continous'] ['rayleigh'] 

area_mean 	 = ['continous'] ['rayleigh'] 

smoothness_mean 	 = ['continous'] ['rayleigh'] 

compactness_mean 	 = ['continous'] ['rayleigh'] 

concavity_mean 	 = ['continous'] ['rayleigh'] 

concave points_mean 	 = ['continous'] ['rayleigh'] 

symmetry_mean 	 = ['continous'] ['rayleigh'] 

fractal_dimension_mean 	 = ['continous'] ['rayleigh'] 

radius_se 	 = ['continous'] ['rayleigh'] 

texture_se 	 = ['continous'] ['rayleigh'] 

perimeter_se 	 = ['continous'] ['rayleigh'] 

area_se 	 = ['continous'] ['rayleigh'] 

smoothness_se 	 = ['continous'] ['rayleigh'] 

compactness_se 	 = ['continous'] ['rayleigh'] 

concavity_se 	 = ['continous'] ['rayleigh'] 

concave points_se 	 = ['continous'] ['rayleigh'] 

symmetry_se 	 = ['continous'] ['rayleigh'] 

fractal_dimension_se 	 = ['continous'] ['rayleigh'] 

radius_